In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Objective & Context
This notebook analyzes the volatility and risk profile of the VFV stock. 
VFV tracks the S&P 500 index, so understanding its risk metrics is crucial for portfolio management.
We will explore historical price data, calculate key risk indicators, and visualize market behavior.

In [14]:
# Load Data
vfv_data = pd.read_csv('../data/vfv_raw_data.csv', parse_dates=['Date'], index_col='Date')
vfv_data.sort_index(inplace=True)
vfv_data.rename(columns={'Market price (CAD)': 'Close'}, inplace=True)
vfv_data['Close'] = vfv_data['Close'].replace('[\$,]', '', regex=True).astype(float)
vfv_data

<>:5: SyntaxWarning: invalid escape sequence '\$'
<>:5: SyntaxWarning: invalid escape sequence '\$'
/var/folders/d_/vtbtm_tx56jbs30plxf7r3mh0000gn/T/ipykernel_68869/950173769.py:5: SyntaxWarning: invalid escape sequence '\$'
  vfv_data['Close'] = vfv_data['Close'].replace('[\$,]', '', regex=True).astype(float)


,NAV (CAD),Close
Date,,
2014-11-18,$41.2499,41.245
2014-11-19,$41.3631,41.360
2014-11-20,$41.2772,41.285
2014-11-21,$41.2651,41.245
2014-11-24,$41.5520,41.565
...,...,...
2025-03-03,$150.7359,150.700
2025-03-04,$148.5961,148.600
2025-03-05,$148.8868,148.800
